In [29]:
import numpy as np
import os
from pprint import pprint
from bark.api import text_to_semantic, semantic_to_waveform, generate_audio
from bark.generation import SAMPLE_RATE, generate_text_semantic
from IPython.display import Audio
from scipy.io.wavfile import write as write_wav
from datetime import datetime
import sys

We generate voice lines based on metadata from an old version of the [Mozilla CommonVoice dataset](https://www.kaggle.com/datasets/nickj26/common-voice-corpus-1?resource=download&select=validated.tsv) metadata. Yes, this is just English for now; I will figure out multilingual later.



In [13]:
import pandas as pd

CV_METADATA_PATH = '../datasets/validated.tsv'
df = pd.read_csv(CV_METADATA_PATH, sep="\t")
df.columns

Index(['client_id', 'path', 'sentence', 'up_votes', 'down_votes', 'age',
       'gender', 'accent'],
      dtype='object')

In [15]:
lines = df["sentence"].unique()
lines

array(['To give chalk for cheese', 'Judge may not think so.',
       'I have already described the appearance of that colossal bulk which was embedded in the ground.',
       ..., "How's the forecast for VI",
       'Please look up the Jenny of the Prairie television show.',
       'Find me the creative work The Pickwick Papers'], dtype=object)

There are enough English lines for ~25 hours of audio; _hopefully_ we'll need less than that.

In [24]:
#%pip install torch --force --extra-index-url https://download.pytorch.org/whl/cu118
import torch


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118
  Using cached https://download.pytorch.org/whl/cu118/torch-2.0.0%2Bcu118-cp310-cp310-linux_x86_64.whl (2267.3 MB)
  Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Using cached https://download.pytorch.org/whl/sympy-1.11.1-py3-none-any.whl (6.5 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached https://download.pytorch.org/whl/Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Using cached filelock-3.12.0-py3-none-any.whl (10 kB)
  Using cached https://download.pytorch.org/whl/triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Using cached cmake-3.26.3-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (24.0 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 11.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached https://download.pytorch.org/whl/MarkupSafe-2.1.2-cp310-cp310-manylinux_2_17_x86_64.ma

In [26]:
#generate_text_semantic = torch.compile(generate_text_semantic)
#semantic_to_waveform = torch.compile(semantic_to_waveform)

On an RTX 4090 with a "large" model there's an RTF of approximately 0.75 with Torch 2.0+cu118, before additional optimizations.

In [33]:
minutes_generated = 0
for i, line in enumerate(lines):
    semantic_tokens = generate_text_semantic(text=line, temp=1)
    waveform_arr = semantic_to_waveform(semantic_tokens)

    semantic_tokens_filepath = f"../datasets/en/{i}_{line}.npz"
    np.savez(semantic_tokens_filepath, tokens=semantic_tokens)
    wav_filepath = f"../datasets/en{i}_{line}.wav"
    write_wav(wav_filepath, SAMPLE_RATE, waveform_arr)

    minutes_generated += len(semantic_tokens) / 49.9 / 60
    print(f"Minutes of audio: {minutes_generated}")

    if minutes_generated > 60:
       break

100%|██████████| 8/8 [00:02<00:00,  3.19it/s]


Minutes of audio: 0.050100200400801605


100%|██████████| 4/4 [00:01<00:00,  3.17it/s]


Minutes of audio: 0.07648630594522378


100%|██████████| 14/14 [00:05<00:00,  2.76it/s]


...

Minutes of audio: 0.9061456245824985


100%|██████████| 11/11 [00:03<00:00,  3.00it/s]


Minutes of audio: 0.9772879091516368


100%|██████████| 19/19 [00:07<00:00,  2.56it/s]


Minutes of audio: 1.1012024048096194
